# forecasting selling price of chicken with Bins

In [3]:
# libraries
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

!pip install category_encoders
import category_encoders as ce
from category_encoders import TargetEncoder




In [4]:
# reading file
df = pd.read_csv('/workspaces/forecasting/data/data.csv')
df.head()

,DAY_DATE,UNIT,TAILS,KG,SALES PER KG,TOTAL SALES,ABW,CHICKEN SIZE,PROVINCE
0,2017-01-01,LOMBOK,6237,11356.5,22000.00000,249843000,1.820827,MEDIUM,NUSA TENGGARA BARAT
1,2017-01-01,GARUT,2625,5446.8,17000.00000,92595600,2.074971,LARGE,JAWA BARAT
2,2017-01-01,SUMEDANG,7788,14176.8,16106.05355,228332300,1.820339,MEDIUM,JAWA BARAT
3,2017-01-01,CIREBON,7504,15958.0,17000.00000,271286000,2.126599,LARGE,JAWA BARAT
4,2017-01-01,PEKALONGAN,9121,16250.0,16500.00000,268125000,1.781603,MEDIUM,JAWA TENGAH


# cleaning

In [5]:
# removing outlier
df = df[df['SALES PER KG'] <= 35000]

# removing 'total sales' column
df = df.drop('TOTAL SALES', axis = 1)

# creating bins for prediction

# using sturges to print out the number of bins
sturges = int(np.ceil(1 + np.log2(len(df['SALES PER KG']))))
print(sturges)

max_value = 35000
bin_width = int((max_value - 0) // sturges)

# Correctly calculate 'bin_edges'
# Ensure 'bin_width' is added to 'max_value' to include the upper edge
bin_edges = np.arange(0, max_value + bin_width, bin_width)

# Use pd.cut to bin the data
df['price_bin'] = pd.cut(df['SALES PER KG'], bins=bin_edges, labels=False, right=False)

# Add 1 to change bins from 0-19 to 1-20
df['price_bin'] += 1

# Display the first few rows to verify
print(df.head())

# dropping sales per kg column as we already have price_bin
df = df.drop('SALES PER KG', axis = 1)


20
     DAY_DATE        UNIT  TAILS       KG  SALES PER KG       ABW  \
0  2017-01-01      LOMBOK   6237  11356.5   22000.00000  1.820827   
1  2017-01-01       GARUT   2625   5446.8   17000.00000  2.074971   
2  2017-01-01    SUMEDANG   7788  14176.8   16106.05355  1.820339   
3  2017-01-01     CIREBON   7504  15958.0   17000.00000  2.126599   
4  2017-01-01  PEKALONGAN   9121  16250.0   16500.00000  1.781603   

  CHICKEN SIZE             PROVINCE  price_bin  
0       MEDIUM  NUSA TENGGARA BARAT         13  
1        LARGE           JAWA BARAT         10  
2       MEDIUM           JAWA BARAT         10  
3        LARGE           JAWA BARAT         10  
4       MEDIUM          JAWA TENGAH         10  


## pre processing
- extracting date column
- splitting, scaling
- creating function to evaluate the model [call `results(y_test, predictions)`]


In [6]:
# extracting year and month from date column for seasonality trends
df['DAY_DATE'] = pd.to_datetime(df['DAY_DATE'], errors='coerce').dt.normalize()
df['YEAR'] = df['DAY_DATE'].dt.year
df['MONTH'] = df['DAY_DATE'].dt.month
df['DAY'] = df['DAY_DATE'].dt.day
df = df.drop('DAY_DATE', axis = 1)


In [7]:
# train test split before label encoding to prevent data leakage


X = df.drop('price_bin', axis=1)
y = df['price_bin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaling numerical variable

scaler = StandardScaler()
columns_to_scale = ['KG', 'TAILS']
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])

In [8]:
# creating function to evaluate the model
def results(y_test, predictions):
  mae = mean_absolute_error(y_test, predictions)
  mse = mean_squared_error(y_test, predictions)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, predictions)

  print(f"Mean Absolute Error (MAE): {mae}")
  print(f"Mean Squared Error (MSE): {mse}")
  print(f"Root Mean Squared Error (RMSE): {rmse}")
  print(f"R-squared (R²): {r2}")


# lazy cat

In [9]:
!pip install dirty_cat
from dirty_cat import SuperVectorizer


In [10]:
super_encoder = SuperVectorizer(auto_cast=True)
X_train_encoded = super_encoder.fit_transform(X_train)
X_test_encoded = super_encoder.transform(X_test)

/home/codespace/.python/current/lib/python3.10/site-packages/sklearn/utils/deprecation.py:66: FutureWarning: Class SuperVectorizer is deprecated; use TableVectorizer instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
X_train_encoded

array([[0.000e+00, 0.000e+00, 1.000e+00, ..., 2.021e+03, 1.000e+01,
        3.000e+01],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.019e+03, 4.000e+00,
        1.200e+01],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.022e+03, 2.000e+00,
        1.500e+01],
       ...,
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.019e+03, 5.000e+00,
        1.600e+01],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.019e+03, 8.000e+00,
        1.600e+01],
       [0.000e+00, 0.000e+00, 1.000e+00, ..., 2.019e+03, 3.000e+00,
        1.700e+01]])

# encoding

In [53]:
# encoding of columns
# label encoding for province

label_encoder = LabelEncoder()
X_train['SIZE ENCODE'] = label_encoder.fit_transform(X_train['CHICKEN SIZE'])
X_test['SIZE ENCODE'] = label_encoder.transform(X_test['CHICKEN SIZE'])

# dropping chicken size column
X_train = X_train.drop(['CHICKEN SIZE'], axis = 1)
X_test = X_test.drop(['CHICKEN SIZE'], axis = 1)


# one hot encoding for province

encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_train[['PROVINCE']])
X_train_encoded = encoder.transform(X_train[['PROVINCE']])
X_test_encoded = encoder.transform(X_test[['PROVINCE']])

# Convert the encoded matrices back to dataframes for easier manipulation
columns = encoder.get_feature_names_out(['PROVINCE'])
X_train_encoded_df = pd.DataFrame(X_train_encoded.toarray(), columns=columns, index=X_train.index)
X_test_encoded_df = pd.DataFrame(X_test_encoded.toarray(), columns=columns, index=X_test.index)

# merge back the dataframe
X_train = pd.concat([X_train.drop('PROVINCE', axis=1), X_train_encoded_df], axis=1)
X_test = pd.concat([X_test.drop('PROVINCE', axis=1), X_test_encoded_df], axis=1)


# one hot encoding for unit
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_train[['UNIT']])
X_train_encoded_unit = encoder.transform(X_train[['UNIT']])
X_test_encoded_unit = encoder.transform(X_test[['UNIT']])

# Convert the encoded matrices back to dataframes for easier manipulation
columns = encoder.get_feature_names_out(['UNIT'])
X_train_encoded_df_unit = pd.DataFrame(X_train_encoded_unit.toarray(), columns=columns, index=X_train.index)
X_test_encoded_df_unit = pd.DataFrame(X_test_encoded_unit.toarray(), columns=columns, index=X_test.index)

# merge back the dataframe, dropping unit column
X_train = pd.concat([X_train.drop('UNIT', axis=1), X_train_encoded_df_unit], axis=1)
X_test = pd.concat([X_test.drop('UNIT', axis=1), X_test_encoded_df_unit], axis=1)




In [56]:
X_train.columns

Index(['TAILS', 'KG', 'ABW', 'YEAR', 'MONTH', 'DAY', 'SIZE ENCODE',
       'PROVINCE_ACEH', 'PROVINCE_BALI', 'PROVINCE_BANTEN',
       ...
       'UNIT_TASIKMALAYA', 'UNIT_TEBING TINGGI', 'UNIT_TEGAL',
       'UNIT_TEMANGGUNG', 'UNIT_TULANG BAWANG', 'UNIT_TULUNGAGUNG',
       'UNIT_UNGARAN', 'UNIT_WABIN', 'UNIT_WONOGIRI', 'UNIT_YOGYAKARTA'],
      dtype='object', length=164)

# trying lazy predict

In [55]:
pip install lazypredict-nightly

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [14]:
# libraries
import lazypredict
from lazypredict.Supervised import LazyRegressor


In [15]:
# initialising regressor
reg = LazyRegressor(verbose=10, ignore_warnings=True, custom_metric=None)

# fitting and evaluate models
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
top5_models = models.sort_values(by='R-Squared', ascending = False).head(5)
print(top5_models)

  0%|          | 0/42 [00:00<?, ?it/s]

  2%|▏         | 1/42 [00:12<08:39, 12.68s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.1541600505980727, 'Adjusted R-Squared': 0.15406063228638012, 'RMSE': 1.7452790101184326, 'Time taken': 12.682361841201782}


  5%|▍         | 2/42 [00:28<09:45, 14.63s/it]

{'Model': 'BaggingRegressor', 'R-Squared': 0.8469774129283023, 'Adjusted R-Squared': 0.8469594269638556, 'RMSE': 0.7423326098238878, 'Time taken': 15.985158681869507}


...

{'Model': 'LinearRegression', 'R-Squared': 0.08775463740416312, 'Adjusted R-Squared': 0.0876474139349207, 'RMSE': 1.812494098367702, 'Time taken': 0.4091050624847412}


 60%|█████▉    | 25/42 [03:32<04:33, 16.09s/it]

{'Model': 'LinearSVR', 'R-Squared': 0.07914069417129699, 'Adjusted R-Squared': 0.07903245823664218, 'RMSE': 1.8210312961631099, 'Time taken': 51.4101722240448}


 62%|██████▏   | 26/42 [06:53<19:02, 71.42s/it]

{'Model': 'MLPRegressor', 'R-Squared': 0.42493648786933724, 'Adjusted R-Squared': 0.42486889607075584, 'RMSE': 1.4390592493675831, 'Time taken': 200.5052525997162}


'ExtraTreesRegressor', 
'BaggingRegressor', 
